In [38]:
import xml.sax
import pandas as pd

element = {}  # Stores the name and the attributes of the current element
publication = {}  # Stores the publication under construction
data = []

class SOLHandler(xml.sax.ContentHandler):

    def startElement(self, name, attrs):
        global element
        element = {
            'name': name,
            'attrs': attrs
        }

    def characters(self, content):
        global element, publication

        if element:
            if element['name'] == 'dc:title' and (element['attrs'].getValue('xml:lang') == 'pt-BR'):
                publication['title'] = content
            elif element['name'] == 'dc:creator':
                author = ' '.join(reversed(content.split(', ')))
                publication['authors'] = f'{publication["authors"]}; {author}' if 'authors' in publication else author
            elif element['name'] == 'dc:date':
                publication['year'] = int(content[0:4])

    def endElement(self, name):
        global element, publication, data

        if name == 'oai_dc:dc':
            data.append(publication)
            publication = {}

        element = None

In [39]:
parser = xml.sax.make_parser()
parser.setContentHandler(SOLHandler())
parser.parse(open('sol.xml', 'r'))

In [41]:
df = pd.DataFrame(data)
df.to_excel('sol.xlsx', index=False)